# Assignment 2

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./05_src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [1]:
# Load the libraries as required.
# Load environment variables
%load_ext dotenv
%dotenv 
# Add src to path
import os
import sys
sys.path.append(os.getenv('SRC_DIR'))

# Standard libraries
import pandas as pd
import numpy as np

In [2]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


In [3]:
fires_dt.head()

,coord_x,coord_y,month,day,ffmc,dmc,dc,isi,temp,rh,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


# Get X and Y

Create the features data frame and target data.

In [4]:
X = fires_dt.drop(columns = 'area')
Y = fires_dt['area']

In [5]:
X

,coord_x,coord_y,month,day,ffmc,dmc,dc,isi,temp,rh,wind,rain
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
512,4,3,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0
513,2,4,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0
514,7,4,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0
515,1,4,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0


In [6]:
Y

0       0.00
1       0.00
2       0.00
3       0.00
4       0.00
       ...  
512     6.44
513    54.29
514    11.16
515     0.00
516     0.00
Name: area, Length: 517, dtype: float64

In [7]:
from sklearn.model_selection import train_test_split

scoring = ['neg_mean_squared_error', 'neg_mean_absolute_error', 'max_error']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import skew, kurtosis

In [9]:

categorical = ['month', 'day']
numeric = ['coord_x', 'coord_y','ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
numeric_power = ['ffmc', 'isi', 'rain']
numeric_standard = list(set(numeric) - set(numeric_power))

In [10]:
preproc1 =  ColumnTransformer([
    ('scale_numeric', StandardScaler(), numeric),
    ('encode_cat', OneHotEncoder(handle_unknown = 'ignore'), categorical)
])
preproc1



ColumnTransformer(transformers=[('scale_numeric', StandardScaler(),
                                 ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                  'isi', 'temp', 'rh', 'wind', 'rain']),
                                ('encode_cat',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['month', 'day'])])

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [11]:
preproc2 = ColumnTransformer([
    ('scale_numeric', StandardScaler(), numeric_standard),
    ('non_linear_scale_numeric', PowerTransformer(), numeric_power),
    ('encode_cat', OneHotEncoder(handle_unknown = 'ignore'), categorical)
])
preproc2

ColumnTransformer(transformers=[('scale_numeric', StandardScaler(),
                                 ['dmc', 'coord_x', 'temp', 'rh', 'coord_y',
                                  'dc', 'wind']),
                                ('non_linear_scale_numeric', PowerTransformer(),
                                 ['ffmc', 'isi', 'rain']),
                                ('encode_cat',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['month', 'day'])])

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [12]:
# Pipeline A = preproc1 + baseline
pipeline_A = Pipeline([
    ('preprocessing', preproc1),
    ('regressor', KNeighborsRegressor() )
])
pipeline_A

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('scale_numeric',
                                                  StandardScaler(),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('encode_cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['month', 'day'])])),
                ('regressor', KNeighborsRegressor())])

In [13]:
pipeline_A.get_params()

{'memory': None,
 'steps': [('preprocessing',
   ColumnTransformer(transformers=[('scale_numeric', StandardScaler(),
                                    ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                     'isi', 'temp', 'rh', 'wind', 'rain']),
                                   ('encode_cat',
                                    OneHotEncoder(handle_unknown='ignore'),
                                    ['month', 'day'])])),
  ('regressor', KNeighborsRegressor())],
 'verbose': False,
 'preprocessing': ColumnTransformer(transformers=[('scale_numeric', StandardScaler(),
                                  ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                   'isi', 'temp', 'rh', 'wind', 'rain']),
                                 ('encode_cat',
                                  OneHotEncoder(handle_unknown='ignore'),
                                  ['month', 'day'])]),
 'regressor': KNeighborsRegressor(),
 'preprocessing__force_int_rema

In [14]:
# Pipeline B = preproc2 + baseline
pipeline_B = Pipeline([
    ('preprocessing', preproc2),
    ('regressor', KNeighborsRegressor() )
])
pipeline_B

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('scale_numeric',
                                                  StandardScaler(),
                                                  ['dmc', 'coord_x', 'temp',
                                                   'rh', 'coord_y', 'dc',
                                                   'wind']),
                                                 ('non_linear_scale_numeric',
                                                  PowerTransformer(),
                                                  ['ffmc', 'isi', 'rain']),
                                                 ('encode_cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['month', 'day'])])),
                ('regressor', KNeighborsRegressor())])

In [15]:
# Pipeline C = preproc1 + advanced model
pipeline_C = Pipeline([
    ('preprocessing', preproc1),
    ('regressor', RandomForestRegressor(random_state=42) )
])
pipeline_C

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('scale_numeric',
                                                  StandardScaler(),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('encode_cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['month', 'day'])])),
                ('regressor', RandomForestRegressor(random_state=42))])

In [16]:
# Pipeline D = preproc2 + advanced model
pipeline_D = Pipeline([
    ('preprocessing', preproc2),
    ('regressor', RandomForestRegressor(random_state=42) )
])
pipeline_D
    

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('scale_numeric',
                                                  StandardScaler(),
                                                  ['dmc', 'coord_x', 'temp',
                                                   'rh', 'coord_y', 'dc',
                                                   'wind']),
                                                 ('non_linear_scale_numeric',
                                                  PowerTransformer(),
                                                  ['ffmc', 'isi', 'rain']),
                                                 ('encode_cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['month', 'day'])])),
                ('regressor', RandomForestRegressor(random_state=42))])

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

### Parameters Baseline model

In [17]:
param_grid_knn = {
    'regressor__n_neighbors': [3, 5, 10, 15],  # Number of neighbors
    'regressor__weights': ['uniform', 'distance'],  # Weighting scheme
    'regressor__p': [1, 2]  # Distance metric: 1 = Manhattan, 2 = Euclidean
}

#### Pipeline A

In [18]:
grid_knn_A = GridSearchCV(
    estimator=pipeline_A,
    param_grid=param_grid_knn,
    scoring=scoring,
    cv=5,
    refit = "neg_mean_absolute_error"
)

In [19]:
grid_knn_A.fit(X_train, Y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('scale_numeric',
                                                                         StandardScaler(),
                                                                         ['coord_x',
                                                                          'coord_y',
                                                                          'ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('encode_cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('regressor', KNeighborsRegressor())]),
             param_grid={'regressor__n_neighbors': [3, 5, 10, 15],
                         'regressor__p': [1, 2],
                         'regressor__weights': ['uniform', 'distance']},
             refit='neg_mean_absolute_error',
             scoring=['neg_mean_squared_error', 'neg_mean_absolute_error',
                      'max_error'])

In [20]:
res = grid_knn_A.cv_results_
res = pd.DataFrame(res)
res.columns

res.sort_values('rank_test_neg_mean_absolute_error')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__n_neighbors,param_regressor__p,param_regressor__weights,params,split0_test_neg_mean_squared_error,split1_test_neg_mean_squared_error,...,std_test_neg_mean_absolute_error,rank_test_neg_mean_absolute_error,split0_test_max_error,split1_test_max_error,split2_test_max_error,split3_test_max_error,split4_test_max_error,mean_test_max_error,std_test_max_error,rank_test_max_error
15,0.003692,0.000051,0.003453,0.000071,15,2,distance,"{'regressor__n_neighbors': 15, 'regressor__p':...",-1598.605473,-477.698510,...,3.312584,1,-270.181768,-87.693211,-149.317345,-724.635225,-190.132699,-284.392050,227.963339,2
14,0.003610,0.000041,0.003247,0.000038,15,2,uniform,"{'regressor__n_neighbors': 15, 'regressor__p':...",-1588.226587,-470.441240,...,3.366404,2,-269.964000,-78.143333,-149.201333,-722.238667,-190.282000,-281.965867,228.718238,1
11,0.003722,0.000085,0.003416,0.000110,10,2,distance,"{'regressor__n_neighbors': 10, 'regressor__p':...",-1694.574423,-569.384672,...,3.820825,3,-273.667581,-124.439466,-163.813639,-718.954862,-187.803087,-293.735727,218.163581,7
10,0.003667,0.000093,0.003152,0.000042,10,2,uniform,"{'regressor__n_neighbors': 10, 'regressor__p':...",-1693.860310,-550.888719,...,3.807611,4,-273.564000,-115.593000,-143.778000,-713.822000,-187.537000,-286.858800,220.065097,3
13,0.003649,0.000075,0.003757,0.000140,15,1,distance,"{'regressor__n_neighbors': 15, 'regressor__p':...",-1736.062972,-476.859364,...,4.073406,5,-275.092126,-74.986818,-157.402273,-743.349662,-190.493051,-288.264786,236.409629,5
12,0.003707,0.000062,0.003608,0.000092,15,1,uniform,"{'regressor__n_neighbors': 15, 'regressor__p':...",-1742.702978,-474.296643,...,4.247264,6,-275.056000,-68.446667,-159.006667,-742.618667,-189.850667,-286.995733,237.199020,4
9,0.003806,0.000201,0.003593,0.000131,10,1,distance,"{'regressor__n_neighbors': 10, 'regressor__p':...",-1761.552174,-623.706162,...,4.137663,7,-274.482316,-103.245179,-197.441217,-745.906581,-189.018130,-302.018685,228.478093,8
8,0.003719,0.000132,0.003469,0.000150,10,1,uniform,"{'regressor__n_neighbors': 10, 'regressor__p':...",-1804.620435,-627.367408,...,4.419657,8,-274.366000,-99.514000,-151.247000,-745.823000,-187.537000,-291.697400,234.111341,6
1,0.003832,0.000134,0.003493,0.000118,3,1,distance,"{'regressor__n_neighbors': 3, 'regressor__p': ...",-1823.720130,-806.437878,...,3.866243,9,-277.321809,-170.267785,-429.707946,-746.280000,-197.838981,-364.283304,211.239152,14
7,0.003676,0.000104,0.003250,0.000089,5,2,distance,"{'regressor__n_neighbors': 5, 'regressor__p': ...",-1964.407651,-1184.287081,...,3.758926,10,-276.370606,-190.783537,-275.606750,-746.118137,-182.185428,-334.212892,209.824055,11


#### Pipeline B

In [22]:
grid_knn_B = GridSearchCV(
    estimator=pipeline_B,
    param_grid=param_grid_knn,
    scoring=scoring,
    cv=5,
    refit = "neg_mean_absolute_error"
)
grid_knn_B.fit(X_train, Y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('scale_numeric',
                                                                         StandardScaler(),
                                                                         ['dmc',
                                                                          'coord_x',
                                                                          'temp',
                                                                          'rh',
                                                                          'coord_y',
                                                                          'dc',
                                                                          'wind']),
                                                                        ('non_linear_scale_numeric',
                                                                         PowerTransformer(),
                                                                         ['ffmc',
                                                                          'isi',
                                                                          'rain']),
                                                                        ('encode_cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('regressor', KNeighborsRegressor())]),
             param_grid={'regressor__n_neighbors': [3, 5, 10, 15],
                         'regressor__p': [1, 2],
                         'regressor__weights': ['uniform', 'distance']},
             refit='neg_mean_absolute_error',
             scoring=['neg_mean_squared_error', 'neg_mean_absolute_error',
                      'max_error'])

### Parameter Advanced Model (Random Forest)

In [31]:
param_grid_rf = {
    'regressor__n_estimators': [100, 200],  
    'regressor__max_depth': [None, 10], 
    'regressor__min_samples_split': [2, 5],  
    'regressor__min_samples_leaf': [1, 2], 
    'regressor__max_features': ['sqrt'], 
    'regressor__bootstrap': [True] 
}

#### Pipeline C

In [32]:
grid_rf_C = GridSearchCV(
    estimator=pipeline_C,
    param_grid=param_grid_rf,
    scoring=scoring,  
    cv=5,                             
    n_jobs=-1,                        
    verbose=1,                        
    refit='neg_mean_absolute_error'
)

grid_rf_C.fit(X_train, Y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('scale_numeric',
                                                                         StandardScaler(),
                                                                         ['coord_x',
                                                                          'coord_y',
                                                                          'ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('encode_cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('regressor',
                                        RandomForestRegressor(random_state=42))]),
             n_jobs=-1,
             param_grid={'regressor__bootstrap': [True],
                         'regressor__max_depth': [None, 10],
                         'regressor__max_features': ['sqrt'],
                         'regressor__min_samples_leaf': [1, 2],
                         'regressor__min_samples_split': [2, 5],
                         'regressor__n_estimators': [100, 200]},
             refit='neg_mean_absolute_error',
             scoring=['neg_mean_squared_error', 'neg_mean_absolute_error',
                      'max_error'],
             verbose=1)

#### Pipeline D

In [33]:
grid_rf_D = GridSearchCV(
    estimator=pipeline_D,
    param_grid=param_grid_rf,
    scoring=scoring,  
    cv=5,                              
    n_jobs=-1,                       
    verbose=1,                         
    refit='neg_mean_absolute_error'
)

grid_rf_D.fit(X_train, Y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('scale_numeric',
                                                                         StandardScaler(),
                                                                         ['dmc',
                                                                          'coord_x',
                                                                          'temp',
                                                                          'rh',
                                                                          'coord_y',
                                                                          'dc',
                                                                          'wind']),
                                                                        ('non_linear_scale_numeric',
                                                                         PowerTransformer(),
                                                                         ['ffmc',
                                                                          'isi',
                                                                          'rain']),
                                                                        ('encode_cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('regressor',
                                        RandomForestRe...))]),
             n_jobs=-1,
             param_grid={'regressor__bootstrap': [True],
                         'regressor__max_depth': [None, 10],
                         'regressor__max_features': ['sqrt'],
                         'regressor__min_samples_leaf': [1, 2],
                         'regressor__min_samples_split': [2, 5],
                         'regressor__n_estimators': [100, 200]},
             refit='neg_mean_absolute_error',
             scoring=['neg_mean_squared_error', 'neg_mean_absolute_error',
                      'max_error'],
             verbose=1)

# Evaluate

+ Which model has the best performance?

In [34]:
print("Pipeline A: Best KNN Parameters:", grid_knn_A.best_params_)
print("Pipeline A: Best KNN Score:", grid_knn_A.best_score_)

print("Pipeline B: Best KNN Parameters:", grid_knn_B.best_params_)
print("Pipeline B: Best KNN Score:", grid_knn_B.best_score_) 

print("Pipeline C: Best RF Parameters:", grid_rf_C.best_params_)
print("Pipeline C: Best RF Score:", grid_rf_C.best_score_)

print("Pipeline D: Best RF Parameters:", grid_rf_D.best_params_)
print("Pipeline D: Best RF Score:", grid_rf_D.best_score_)

Pipeline A: Best KNN Parameters: {'regressor__n_neighbors': 15, 'regressor__p': 2, 'regressor__weights': 'distance'}
Pipeline A: Best KNN Score: -15.797052585845696
Pipeline B: Best KNN Parameters: {'regressor__n_neighbors': 15, 'regressor__p': 2, 'regressor__weights': 'distance'}
Pipeline B: Best KNN Score: -15.914328621832604
Pipeline C: Best RF Parameters: {'regressor__bootstrap': True, 'regressor__max_depth': 10, 'regressor__max_features': 'sqrt', 'regressor__min_samples_leaf': 2, 'regressor__min_samples_split': 2, 'regressor__n_estimators': 200}
Pipeline C: Best RF Score: -17.241071688053182
Pipeline D: Best RF Parameters: {'regressor__bootstrap': True, 'regressor__max_depth': 10, 'regressor__max_features': 'sqrt', 'regressor__min_samples_leaf': 2, 'regressor__min_samples_split': 5, 'regressor__n_estimators': 100}
Pipeline D: Best RF Score: -17.48096266895542


Based on the errors, output given by pipeline A (model KNN using standard scaler for all numeric features) has the lowest error and is the best parameters. Possibly, one way to improve the other models (from pipeline B to D) would involve a different non-linear transformation approach and using a different advanced model and playing with their parameters (for pipeline C and D).

# Export

+ Save the best performing model to a pickle file.

In [35]:
import pickle

In [36]:
pipe_best = pipeline_A
model = 'KNeighborsRegressor'

artifacts_dir = os.getenv('ARTIFACTS_DIR')
os.makedirs(artifacts_dir, exist_ok=True)

outpath = os.path.join(
        artifacts_dir, 
        f"model_{model}.pkl")

with open(outpath, 'wb') as f:
        pickle.dump(pipe_best, f)

In [37]:
model_pkl_file = os.path.join(
    os.getenv('ARTIFACTS_DIR'), 
    "model_KNeighborsRegressor.pkl"
    )

with open(model_pkl_file, 'rb') as file:
    pipe = pickle.load(file)
pipe

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('scale_numeric',
                                                  StandardScaler(),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('encode_cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['month', 'day'])])),
                ('regressor', KNeighborsRegressor())])

In [38]:
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
pipe.fit(X_train, Y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('scale_numeric',
                                                  StandardScaler(),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('encode_cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['month', 'day'])])),
                ('regressor', KNeighborsRegressor())])

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

*(Answer here.)*

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at the `help` channel. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.